<a href="https://colab.research.google.com/github/StanHillServian/hashtag-london-stream/blob/main/hashtag_london_stream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys

from tweepy import OAuthHandler
from tweepy import API
from tweepy import Stream
from tweepy.streaming import StreamListener
import json

# Replace the "None"s by your own credentials
ACCESS_TOKEN = '1319427930439749633-MLh2G2POhQbcCIEHXMHmRcVQxrMizR'
ACCESS_TOKEN_SECRET = 'bPb6zZAN7zICHbAhwbq23Ju24igdsf0ALOBTstE1Qfr8C'
CONSUMER_KEY = 'lbRF9YBBJvAaDPrT1PbyjIVKD'
CONSUMER_SECRET = 'OL0Gg0XrDtMdHcwvyocPw7ZwvZJvlhZcMWAVi28vWiUOjejWP3'

auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = API(auth, wait_on_rate_limit=True,
          wait_on_rate_limit_notify=True)

class Listener(StreamListener):
    def __init__(self, output_file=sys.stdout):
        super(Listener,self).__init__()
        self.output_file = output_file
    def on_status(self, status):
      data = {}
      data['created_at'] = str(status.created_at)
      data['tweet'] = status.text
      data['handle'] = status.user.screen_name
      json.dump(data, self.output_file)
        #print([status.created_at, status.text, status.user.screen_name, file=self.output_file)
    def on_error(self, status_code):
        print(status_code)
        return False


output = open('stream_output.json', 'w')
listener = Listener(output_file=output)

stream = Stream(auth=api.auth, listener=listener)
try:
    print('Start streaming.')
    stream.filter(track=['#london'])
except KeyboardInterrupt:
    print("Stopped.")
finally:
    print('Done.')
    stream.disconnect()
    output.close()

In [ ]:
#!/usr/bin/env python
# Copyright 2015 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""This script uses the Twitter Streaming API, via the tweepy library,
to pull in tweets and publish them to a PubSub topic.
"""

import base64
import datetime
import os
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
import json 

##import utils

# Get your twitter credentials from the environment variables.
# These are set in the 'twitter-stream.json' manifest file.

consumer_key = 'lbRF9YBBJvAaDPrT1PbyjIVKD'
consumer_secret = 'OL0Gg0XrDtMdHcwvyocPw7ZwvZJvlhZcMWAVi28vWiUOjejWP3'
access_token = '1319427930439749633-MLh2G2POhQbcCIEHXMHmRcVQxrMizR'
access_token_secret = 'bPb6zZAN7zICHbAhwbq23Ju24igdsf0ALOBTstE1Qfr8C'

#PUBSUB_TOPIC = os.environ['PUBSUB_TOPIC']
NUM_RETRIES = 3

class newdict(dict):
        def __str__(self):
            return json.dumps(self)

def publish(data_lines):

    messages = []
    """Publish to the given pubsub topic."""
    #for line in data_lines:
     #   print("PRINTING LINE")
     #   print(type(line))
     #   print(line)
    #    res = json.loads(line)
    #    message = str(cleanup(res))
        #print("PRINTING MESSAGE CLEANED UP")
        #print(type(message))
        #print(message)
    
    for line in data_lines:
      print(line)
      data = json.loads(line)
      try:
        user = data["user"]
        clean_data=json.dumps({
              "text": data["text"],
              "user_handle": user["screen_name"],
              "id": data["id"],
              "posted_at": data["created_at"]
           }).encode('utf-8')
        pub = base64.urlsafe_b64encode(clean_data)
        messages.append({'data': pub})
     
      except Exception as e:
        print(e)
        raise


        
    body = {'messages': messages}
    print(body)

    #body = {'messages': messages}
    #print("publish message: %s" % line)
    #resp = client.projects().topics().publish(
            #topic=pubsub_topic, body=body).execute(num_retries=NUM_RETRIES)
    #return resp


def cleanup(data):
  if isinstance(data, dict):
    newlist = []

    for k, v in data.items():
      if (k == 'retweeted_status') and isinstance(v, dict):
        for a, b in v.items():
          if (a == 'extended_tweet') and isinstance(b, dict):
            for c, d in b.items():
              if (c == 'full_text') and isinstance(d, str):
                newlist.append([c, d])
      if (k == 'text') and isinstance(v, str):
        newlist.append([k, v])
      elif k == 'created_at' and v:
        newlist.append([k, v])
      elif k == 'user' and isinstance(v, dict):
        for a, b in v.items():
         if (a == 'screen_name') and isinstance(b, str):
          newlist.append([a, b])

    mydict = newdict(newlist)
    return mydict
  else:
    return data
    
 
          


class StdOutListener(StreamListener):
    """A listener handles tweets that are received from the stream.
    This listener dumps the tweets into a PubSub topic
    """

    count = 0
    twstring = ''
    tweets = []
    batch_size = 3
    total_tweets = 10000000
    #client = utils.create_pubsub_client(utils.get_credentials())

    def write_to_pubsub(self, tw):
        publish(tw)

    def on_data(self, data):
        """What to do when tweet data is received."""
        self.tweets.append(data)
        if len(self.tweets) >= self.batch_size:
            self.write_to_pubsub(self.tweets)
            self.tweets = []
        self.count += 1
        # if we've grabbed more than total_tweets tweets, exit the script.
        # If this script is being run in the context of a kubernetes
        # replicationController, the pod will be restarted fresh when
        # that happens.
        if self.count > self.total_tweets:
            return False
        if (self.count % 1000) == 0:
            print("count is: %s at %s" % (self.count, datetime.datetime.now()))
        return True

    def on_error(self, status):
        print(status)


if __name__ == '__main__':
    print('....')
    listener = StdOutListener()
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    #print('stream mode is: %s' % os.environ['TWSTREAMMODE'])

    stream = Stream(auth, listener, tweet_mode='extended')
    # set up the streaming depending upon whether our mode is 'filter', which
    # will stream the #london hashtag. If it is not, it will sample random 
    # This environment var is set in the 'twitter-stream.yaml' file.
    stream.filter(track=['#london'])

In [1]:
import tweepy
from google.cloud import pubsub_v1

# Authenticate
auth = tweepy.OAuthHandler('lbRF9YBBJvAaDPrT1PbyjIVKD', 'OL0Gg0XrDtMdHcwvyocPw7ZwvZJvlhZcMWAVi28vWiUOjejWP3')
auth.set_access_token('1319427930439749633-MLh2G2POhQbcCIEHXMHmRcVQxrMizR', 'bPb6zZAN7zICHbAhwbq23Ju24igdsf0ALOBTstE1Qfr8C')

# Configure to wait on rate limit if necessary
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=False)

# Hashtag
hashtag = "#london"

# Function to write data to
def write_to_pubsub(data):
    try:
      # publish to the topic, don't forget to encode everything at utf8!
      publisher.publish(topic_path, data=json.dumps({
          "text": data["text"],
          "user_id": data["user_id"],
          "id": data["id"],
          "posted_at": datetime.datetime.fromtimestamp(data["created_at"]).strftime('%Y-%m-%d %H:%M:%S')
      }).encode("utf-8"), tweet_id=str(data["id"]).encode("utf-8"))
            
    except Exception as e:
        print(e)
        raise

def cleanup(data):
  if isinstance(data, dict):
    newdict = {}

    for k, v in data.items():
      if (k == 'retweeted_status') and isinstance(v, dict):
        for a, b in v.items():
          if (a == 'extended_tweet') and isinstance(b, dict):
            for c, d in b.items():
              if (c == 'full_text') and isinstance(d, str):
                newdict["full_tweet"] = d
      if (k == 'text') and isinstance(v, str):
        newdict["tweet"] = v
      elif k == 'created_at' and v:
        newdict[k] = v
      elif k == 'user' and isinstance(v, dict):
        for a, b in v.items():
         if (a == 'screen_name') and isinstance(b, str):
          newdict[a] = b
    print(newdict)
    return newdict
  else:
    return data

# Listener class
class TweetListener(StreamListener):

    def __init__(self):
        super().__init__()

    def on_status(self, data):
        # When receiveing a tweet: send it to pubsub
        print(data)
        write_to_pubsub(cleanup(data._json))
        return True

    def on_error(self, status):
        if status == 420:
            print("rate limit active")
            return False
          
# Make an instance of the class
l = TweetListener()

# Start streaming
stream = tweepy.Stream(auth, l, tweet_mode='extended')
stream.filter(track=hashtag)

# Configure the connection
publisher = pubsub_v1.PublisherClient()
topic_path = publisher.topic_path('servian-task', 'hashtag-london-topic-2')


ImportError: ignored